In [ ]:
!pip install tensorflow opencv-python matplotlib 

In [ ]:
import tensorflow as tf
import os
import cv2
import imghdr

In [ ]:
# removing dodgy images

extensions = ['jpeg', 'jpg', 'bmp', 'png']

for folder in os.listdir('pics'):
    if folder != '.DS_Store':
        for image in os.listdir(os.path.join('pics', folder)):
            img_path = os.path.join('pics', folder, image)
            try:
                img = cv2.imread(img_path)
                ext = imghdr.what(img_path)
                if ext not in extensions:
                    print('Image not in extensions list {}'.format(img_path))
                    os.remove(img_path)
            except Exception as e:
                print('Problem with image {}'.format(img_path))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# loading data
data = tf.keras.utils.image_dataset_from_directory('pics')
data_iterator = data.as_numpy_iterator() # in order to loop through our data

In [ ]:
# getting a batch of data
batch = data_iterator.next()

In [ ]:
batch[0] # images represented as numpy arrays
batch[0].shape # taken in batches of 32 with 256x256 dimensions

In [ ]:
batch[1] # images in the batch classified into sad/happy people based on binary values.
         # 1 for sad, 0 for happy

In [ ]:
#visualizing the images
fig, ax = plt.subplots(ncols = 4, figsize = (20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# scaling the data
data = data.map(lambda x, y: (x/255, y))
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()

In [ ]:
# visualizing the scaled images
fig, ax = plt.subplots(ncols = 4, figsize = (20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# splitting data

len(data) # 6 batches

train_size = int(len(data) * .7) # roughly dividing our data by alloting 70% to training the model
val_size = int(len(data) * .2)   # 20% for validation (evaluating the model)
test_size = int(len(data) * .1) + 1 # 10% to testing the data

train_size + val_size + test_size # checking if it equals len(data) 

In [45]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [49]:
# developing the deep learning model
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3, 3), 1, activation = 'relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3, 3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add (Dense(256, activation = 'relu'))
model.add (Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile('adam', loss = tf.losses.BinaryCrossentropy(), metrics = ['accuracy'])
model.summary()

In [ ]:
# training the data
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)
hist = model.fit(train, epochs = 20, validation_data = val, callbacks = [tensorboard_callback])

In [ ]:
# plotting the performance for loss
fig = plt.figure()
plt.plot(hist.history['loss'], color = 'red', label = 'loss') 
plt.plot(hist.history['val_loss'], color = 'orange', label = 'val_loss')
fig.suptitle('Loss', fontsize = 20)
plt.legend (loc = "upper left" )
plt.show()

In [ ]:
# plotting the performance for accuracy
fig = plt.figure()
plt.plot(hist.history['accuracy'], color = 'blue', label = 'accuracy') 
plt.plot(hist.history['val_accuracy'], color = 'teal', label = 'val_accuracy')
fig.suptitle('Accuracy', fontsize = 20)
plt.legend (loc = "upper left" )
plt.show()

In [69]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
# evaluating our model on the test data

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

for batch in test.as_numpy_iterator():
    x, y = batch
    yhat = model.predict(x)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precison: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

In [ ]:
# testing our model on new pictures outside the dataset
img = cv2.imread('sadtest.jpg')
resize = tf.image.resize(img, (256, 256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize / 255, 0))

In [ ]:
if yhat > 0.5:
    print('Predicted class is sad.')
else:
    print('Predicted class is happy.')

In [121]:
from tensorflow.keras.models import load_model

In [ ]:
# saving the model
model.save(os.path.join('models', 'happy_or_sad_classification.h5'))
new_model = load_model(os.path.join('models', 'happy_or_sad_classification.h5'))